<a href="https://colab.research.google.com/github/ak2742/neural-net/blob/main/nn_core.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title import libraries

import math
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#@title define Value

class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op   # operation
    self.label = label

  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)  # check input type
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)  # check input type
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward

    return out

  def __pow__(self, other):   # power raise to 'other'
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out

  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')

    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward

    return out

  def relu(self):
    out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')

    def _backward():
        self.grad += (out.data > 0) * out.grad
    out._backward = _backward

    return out

  def exp(self):    # exponential
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')

    def _backward():
      self.grad += out.data * out.grad
    out._backward = _backward

    return out

  def backward(self):
    # topological order all of the children in the graph
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

  def __rmul__(self, other): # other * self
    return self * other

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other)

  def __radd__(self, other): # other + self
    return self + other

  def __rsub__(self, other): # other - self
    return other + (-self)

  def __rtruediv__(self, other): # other / self
    return other * self**-1

  def __repr__(self):
    return f"Value(data={self.data})"



In [ ]:
#@title fn to visualize the data

# from graphviz import Digraph

# def trace(root):
#   # builds a set of all nodes and edges in a graph
#   nodes, edges = set(), set()
#   def build(v):
#     if v not in nodes:
#       nodes.add(v)
#       for child in v._prev:
#         edges.add((child, v))
#         build(child)
#   build(root)
#   return nodes, edges

# def draw_dot(root):
#   dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right

#   nodes, edges = trace(root)
#   for n in nodes:
#     uid = str(id(n))
#     # for any value in the graph, create a rectangular ('record') node for it
#     dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
#     if n._op:
#       # if this value is a result of some operation, create an op node for it
#       dot.node(name = uid + n._op, label = n._op)
#       # and connect this node to it
#       dot.edge(uid + n._op, uid)

#   for n1, n2 in edges:
#     # connect n1 to the op node of n2
#     dot.edge(str(id(n1)), str(id(n2)) + n2._op)

#   return dot

In [ ]:
#@title a neuron

# # inputs x1,x2
# x1 = Value(2.0, label='x1')
# x2 = Value(0.0, label='x2')
# # weights w1,w2
# w1 = Value(-3.0, label='w1')
# w2 = Value(1.0, label='w2')
# # bias of the neuron
# b = Value(6.8813735870195432, label='b')
# # x1*w1 + x2*w2 + b
# x1w1 = x1*w1; x1w1.label = 'x1*w1'
# x2w2 = x2*w2; x2w2.label = 'x2*w2'
# x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
# n = x1w1x2w2 + b; n.label = 'n'
# # activation fn
# o = n.tanh(); o.label = 'o'

In [ ]:
#@title Create MLP

class Module:
  def zero_grad(self):
    for p in self.parameters():
      p.grad = 0

  def parameters(self):
    return []

class Neuron(Module):

  def __init__(self, nin, nonlin=True):  # nin is no of inputs
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))
    self.nonlin = nonlin
    # self.params = self.w + [self.b]

  def __call__(self, x):  # x is a list of inputs
    # w * x + b
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    # out = act.tanh()
    # return out
    return act.relu() if self.nonlin else act

  def parameters(self):
    return self.w + [self.b]

  def __repr__(self):
    return f"{'ReLU' if self.nonlin else 'Linear'} Neuron({len(self.w)})"

class Layer(Module):

  def __init__(self, nin, nout, **kwargs):  # no of inputs and outputs(neurons) from a layer
    self.neurons = [Neuron(nin, **kwargs) for _ in range(nout)]

  def __call__(self, x):  # x is a list of inputs
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs

  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]

  def __repr__(self):
    return f"Layer of [{', '.join(str(n) for n in self.neurons)}]"

class MLP(Module):

  def __init__(self, nin, nouts):  # no of inputs and a list of outputs from each layer
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1], nonlin=i!=len(nouts)-1) for i in range(len(nouts))] #non linear for inner layers

  def __call__(self, x):  # x is a list of inputs
    for layer in self.layers:
      x = layer(x)
    return x

  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]

  def __repr__(self):
    return f"MLP of [{', '.join(str(layer) for layer in self.layers)}]"

In [ ]:
# x = [2.0, 3.0]
# n = MLP(2, [2, 2, 1]) # MLP: 3->4->4->1
# q = n(x)

In [ ]:
# draw_dot(q)

In [ ]:
# xs = [
#   [2.0, 3.0],
#   [3.0, -1.0],
#   [0.5, 1.0],
#   [1.0, 1.0],
# ]
# ys = [1.0, -1.0, -1.0, 1.0] # desired targets

In [ ]:
#@title loss and optimization

# for k in range(20):

#   # forward pass
#   ypred = [n(x) for x in xs]
#   loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))

#   # backward pass
#   for p in n.parameters():
#     p.grad = 0.0
#   loss.backward()

#   # update
#   for p in n.parameters():
#     p.data += -0.1 * p.grad

#   print(k, loss.data)